In [1]:
from typing import Optional, Callable, TypedDict, Union, Iterable, NamedTuple, List
from dataclasses import dataclass, field
import numpy as np
import pandas as pd
import xarray as xr
import torch
from torch.utils.data import get_worker_info
from torch.utils.data.distributed import DistributedSampler
import torch.utils.data
import datetime
import os
from glob import glob
from itertools import repeat
from timeit import timeit
import yaml

from torchvision import transforms
from credit.transforms404 import NormalizeState, ToTensor, load_transforms
from credit.data import CONUS404Dataset

/glade/work/schreck/conda-envs/holodec/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/glade/work/schreck/conda-envs/holodec/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [42]:
config = "../config/conus404.yml"

In [43]:
with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

In [44]:
conf['data']

{'variables': ['PSFC',
  'Q500',
  'Q850',
  'T2',
  'T500',
  'T850',
  'totalVap',
  'U10',
  'U1000',
  'U250',
  'U500',
  'U850',
  'V10',
  'V1000',
  'V250',
  'V500',
  'V850',
  'Z050',
  'Z1000',
  'Z500',
  'Z850'],
 'static_variables': [],
 'scaler_type': 'std',
 'save_loc': '/glade/derecho/scratch/mcginnis/???*',
 'mean_path': '/glade/derecho/scratch/mcginnis/STAGING/All_2010_staged.mean.Lev.SLO.nc',
 'std_path': '/glade/derecho/scratch/mcginnis/STAGING/All_2010_staged.std.Lev.SLO.nc',
 'history_len': 2,
 'forecast_len': 1,
 'valid_history_len': 2,
 'valid_forecast_len': 1,
 'time_step': 1}

In [97]:
import torch
import logging
import numpy as np
import xarray as xr
from credit.data import Sample
from typing import Dict
from torchvision import transforms as tforms

class ToTensor:
    def __init__(self, conf):
        self.conf = conf
        self.hist_len = int(conf["data"]["history_len"])
        self.for_len = int(conf["data"]["forecast_len"])
        self.variables = conf["data"]["variables"]
        self.static_variables = conf["data"]["static_variables"]
        #self.x = 1016
        #self.y = 1638
        self.slice_x = slice(120, 632, None)
        self.slice_y = slice(300, 812, None)

    def __call__(self, sample: Sample) -> Sample:

        return_dict = {}

        for key, value in sample.items():
            if key == 'historical_ERA5_images' or key == 'x':
                self.datetime = value['Time']
                self.doy = value['Time.dayofyear']
                self.hod = value['Time.hour']

            if isinstance(value, xr.DataArray):
                value_var = value.values

            elif isinstance(value, xr.Dataset):
                concatenated_vars = []
                for vv in self.variables:
                    value_var = value[vv].values
                    if len(value_var.shape) == 4: # some seem to have extra single dimensions
                        value_var = value_var.squeeze(1)
                    concatenated_vars.append(value_var[:, self.slice_x, self.slice_y])
                concatenated_vars = np.array(concatenated_vars)

            else:
                value_var = value

            if key == 'x':
                x = torch.as_tensor(np.vstack([np.expand_dims(x, axis=0) for x in concatenated_vars]))
                return_dict['x'] = x

            elif key == 'y':
                y = torch.as_tensor(np.vstack([np.expand_dims(x, axis=0) for x in concatenated_vars]))
                return_dict['y'] = y

        if self.static_variables:
            pass

        return return_dict


In [98]:
transform = transforms.Compose([
    #NormalizeState(conf["data"]["mean_path"],conf["data"]["std_path"]),
    ToTensor(conf)
])

In [99]:
dataset = CONUS404Dataset(
    zarrpath = "/glade/campaign/ral/risc/DATA/conus404/zarr",
    varnames = conf['data']['variables'],
    history_len = conf['data']['history_len'],
    forecast_len = conf['data']['forecast_len'],
    transform = transform
)

In [100]:
result = dataset.__getitem__(0)

In [103]:
result['x'].shape # (C, T, L, W)

torch.Size([21, 2, 512, 512])

In [102]:
result['y'].shape # (C, T, L, W)

torch.Size([21, 1, 512, 512])